In [2]:
import numpy as np
import pandas as pd 
import requests 
import xlsxwriter 
import math 
from nltk import ne_chunk

In [3]:
stocks = pd.read_csv('sp_500_stocks.csv')
stocks.head()

,Symbol,Name,Sector
0,MMM,3M,Industrials
1,AOS,A. O. Smith,Industrials
2,ABT,Abbott Laboratories,Health Care
3,ABBV,AbbVie,Health Care
4,ABMD,Abiomed,Health Care


In [4]:
from hi import IEX_CLOUD_API_TOKEN

In [5]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
print(data)

{'avgTotalVolume': 86480277, 'calculationPrice': 'close', 'change': -2.1, 'changePercent': -0.01316, 'close': 164.14, 'closeSource': 'fcaoilif', 'closeTime': 1646132867152, 'companyName': 'Apple Inc', 'currency': 'USD', 'delayedPrice': 170.13, 'delayedPriceTime': 1677308709274, 'extendedChange': -0.81, 'extendedChangePercent': -0.00498, 'extendedPrice': 165.58, 'extendedPriceTime': 1682401593582, 'high': 168.4, 'highSource': 'e pn uremi a5le1citddye', 'highTime': 1644412681276, 'iexAskPrice': 0, 'iexAskSize': 0, 'iexBidPrice': 0, 'iexBidSize': 0, 'iexClose': 162.52, 'iexCloseTime': 1643208067512, 'iexLastUpdated': 1691432917216, 'iexMarketPercent': 0.013623066151909436, 'iexOpen': 167.71, 'iexOpenTime': 1691947460832, 'iexRealtimePrice': 169.12, 'iexRealtimeSize': 3, 'iexVolume': 1659280, 'lastTradeTime': 1675752838304, 'latestPrice': 169, 'latestSource': 'Close', 'latestTime': 'January 21, 2022', 'latestUpdate': 1671650938108, 'latestVolume': 127996992, 'low': 164.14, 'lowSource': 'me

In [6]:
price = data['latestPrice']
market_cap = data['marketCap']
print(market_cap/1000000000000)
print(price)

2.779789774205
169


In [7]:
my_columns = ['Ticker', 'Stock Price', 'Market Capitalisation', 'Number of Shares to Buy']
final_dataframe = pd.DataFrame(columns = my_columns)
final_dataframe

,Ticker,Stock Price,Market Capitalisation,Number of Shares to Buy


In [8]:
final_dataframe.append(
    pd.Series(
    [
        symbol,
        price,
        market_cap,
        'N/A'
        
    ],
    index = my_columns),
    
    ignore_index=True
)

,Ticker,Stock Price,Market Capitalisation,Number of Shares to Buy
0,AAPL,169,2779789774205,N/A


In [9]:
final_dataframe = pd.DataFrame(columns = my_columns)
for symbol in stocks['Symbol'][:5]:
    api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(api_url).json()
    final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data['latestPrice'], 
                                                   data['marketCap'], 
                                                   'N/A'], 
                                                  index = my_columns), 
                                        ignore_index = True)

In [10]:
final_dataframe

,Ticker,Stock Price,Market Capitalisation,Number of Shares to Buy
0,MMM,172.73,101660417660,N/A
1,AOS,77.11,12448397638,N/A
2,ABT,129.03,228129336618,N/A
3,ABBV,135.01,236449212202,N/A
4,ABMD,316.28,14337262787,N/A


In [11]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [32]:
symbol_groups = list(chunks(stocks['Symbol'][:100], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'], 
                                                   data[symbol]['quote']['marketCap'], 
                                                   'N/A'], 
                                                  index = my_columns), 
                                        ignore_index = True)
        
    
final_dataframe

,Ticker,Stock Price,Market Capitalisation,Number of Shares to Buy
0,MMM,174.97,102009257940,N/A
1,AOS,78.34,12866151078,N/A
2,ABT,129.16,227403370327,N/A
3,ABBV,135.69,238344426010,N/A
4,ABMD,308.28,14469468882,N/A
...,...,...,...,...
95,CE,166.36,17772126926,N/A
96,CNC,80.86,45761769195,N/A
97,CNP,27.92,17784800084,N/A
98,CDAY,77.22,12127102916,N/A


In [33]:
portfolio_size = input("Enter the value of your portfolio : ")

try :
    val = float(portfolio_size)
    print(val)
except : 
    print("Please enter an integer")
    portfolio_size = input("That's not a number! \nPlease try again : ")
    val = float(portfolio_size)
    

Enter the value of your portfolio : 999999999
999999999.0


In [36]:
position_size = val/len(final_dataframe.index)
for i in range(0, len(final_dataframe.index)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe['Stock Price'][i])
    
final_dataframe

,Ticker,Stock Price,Market Capitalisation,Number of Shares to Buy
0,MMM,174.97,102009257940,57152
1,AOS,78.34,12866151078,127648
2,ABT,129.16,227403370327,77423
3,ABBV,135.69,238344426010,73697
4,ABMD,308.28,14469468882,32438
...,...,...,...,...
95,CE,166.36,17772126926,60110
96,CNC,80.86,45761769195,123670
97,CNP,27.92,17784800084,358166
98,CDAY,77.22,12127102916,129500


In [50]:
writer = pd.ExcelWriter('recommended trades.xlsx', engine = 'xlsxwriter')
final_dataframe.to_excel(writer, 'Recommended Trades', index = False)

In [51]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_format = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_format = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [52]:
writer.sheets['Recommended Trades'].set_column('A:A', 18, string_format)
writer.sheets['Recommended Trades'].set_column('B:B', 18, string_format)
writer.sheets['Recommended Trades'].set_column('C:C', 18, string_format)
writer.sheets['Recommended Trades'].set_column('D:D', 18, string_format)
writer.save()

In [53]:
column_formats = { 
                    'A': ['Symbol', string_format],
                    'B': ['Price', dollar_format],
                    'C': ['Market Capitalization', dollar_format],
                    'D': ['Number of Shares to Buy', integer_format]
                    }

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], string_format)